In [3]:
import tqdm
import torchvision.models.resnet as resnet
import torch
import os
import argparse
from dotenv import load_dotenv, find_dotenv
from src.data.dataloader import UseMetaData, ValTransforms
from torch.utils.data import DataLoader

In [7]:
dotenvpath = find_dotenv()
load_dotenv(dotenvpath)

annotation_path = "../../data/annotations/"
path = '/mnt/f/MetalabelIntegration/'

In [9]:
train_data = UseMetaData(
        "train", path, annotation_path, transform=ValTransforms()
    )
val_data = UseMetaData("val", path, annotation_path, transform=ValTransforms())
    
number_of_classes = len(train_data.classes)

train_loader = DataLoader(
        train_data,
        batch_size=16,
        num_workers=8,
        pin_memory=True,
        shuffle=True,
    )

val_loader = DataLoader(
        val_data,
        batch_size=16,
        num_workers=8,
        pin_memory=True,
        shuffle=True,
    )

In [10]:
model = torch.hub.load(
                "pytorch/vision:v0.9.0",
                "resnet50",
                weights="ResNet50_Weights.IMAGENET1K_V1",
            )
model.fc = torch.nn.Identity()
model.eval()

Using cache found in /home/juliu/.cache/torch/hub/pytorch_vision_v0.9.0


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [11]:
train_img_data = []
train_meta_data = []
train_labels = []

for i, batch in enumerate(tqdm.tqdm(train_loader)):
    train_img_data.append(model(batch[0]))
    train_meta_data.append(batch[1])
    train_labels.append(batch[2])
    if i > 100:
        break

train_img_data = torch.cat(train_img_data, 0).detach()
train_meta_data = torch.cat(train_meta_data, 0)
train_cat_data = torch.cat([train_img_data, train_meta_data], 1)
train_labels = torch.cat(train_labels, 0)

  0%|          | 1/3055 [00:03<3:02:12,  3.58s/it]

[tensor([[0.1735, 1.3912, 0.2394,  ..., 0.2795, 0.1643, 0.5283],
        [0.4992, 0.0203, 0.2935,  ..., 0.3883, 0.1765, 4.6640],
        [0.0753, 0.1756, 0.2339,  ..., 1.1095, 0.0155, 0.3600],
        ...,
        [0.7140, 0.5692, 0.4538,  ..., 0.2592, 0.5171, 0.3044],
        [0.1077, 1.2094, 0.1961,  ..., 1.0288, 0.0437, 0.9602],
        [0.0205, 0.5060, 0.9473,  ..., 0.8195, 0.0560, 0.4882]],
       grad_fn=<ReshapeAliasBackward0>)]


  0%|          | 2/3055 [00:04<1:37:01,  1.91s/it]

[tensor([[0.1735, 1.3912, 0.2394,  ..., 0.2795, 0.1643, 0.5283],
        [0.4992, 0.0203, 0.2935,  ..., 0.3883, 0.1765, 4.6640],
        [0.0753, 0.1756, 0.2339,  ..., 1.1095, 0.0155, 0.3600],
        ...,
        [0.7140, 0.5692, 0.4538,  ..., 0.2592, 0.5171, 0.3044],
        [0.1077, 1.2094, 0.1961,  ..., 1.0288, 0.0437, 0.9602],
        [0.0205, 0.5060, 0.9473,  ..., 0.8195, 0.0560, 0.4882]],
       grad_fn=<ReshapeAliasBackward0>), tensor([[0.4931, 0.8447, 0.2776,  ..., 0.5990, 0.2028, 0.2212],
        [0.2844, 0.1453, 0.0804,  ..., 0.1438, 0.0773, 0.2300],
        [0.4367, 0.6660, 0.0899,  ..., 0.0412, 0.2838, 0.1871],
        ...,
        [0.3342, 0.1397, 0.4592,  ..., 0.2362, 0.0444, 0.6778],
        [0.1249, 0.4872, 0.4672,  ..., 0.4119, 0.0045, 0.0271],
        [0.5475, 0.6624, 0.1561,  ..., 0.4537, 0.2599, 0.2959]],
       grad_fn=<ReshapeAliasBackward0>)]


  0%|          | 3/3055 [00:05<1:09:14,  1.36s/it]

...

[tensor([[0.1735, 1.3912, 0.2394,  ..., 0.2795, 0.1643, 0.5283],
        [0.4992, 0.0203, 0.2935,  ..., 0.3883, 0.1765, 4.6640],
        [0.0753, 0.1756, 0.2339,  ..., 1.1095, 0.0155, 0.3600],
        ...,
        [0.7140, 0.5692, 0.4538,  ..., 0.2592, 0.5171, 0.3044],
        [0.1077, 1.2094, 0.1961,  ..., 1.0288, 0.0437, 0.9602],
        [0.0205, 0.5060, 0.9473,  ..., 0.8195, 0.0560, 0.4882]],
       grad_fn=<ReshapeAliasBackward0>), tensor([[0.4931, 0.8447, 0.2776,  ..., 0.5990, 0.2028, 0.2212],
        [0.2844, 0.1453, 0.0804,  ..., 0.1438, 0.0773, 0.2300],
        [0.4367, 0.6660, 0.0899,  ..., 0.0412, 0.2838, 0.1871],
        ...,
        [0.3342, 0.1397, 0.4592,  ..., 0.2362, 0.0444, 0.6778],
        [0.1249, 0.4872, 0.4672,  ..., 0.4119, 0.0045, 0.0271],
        [0.5475, 0.6624, 0.1561,  ..., 0.4537, 0.2599, 0.2959]],
       grad_fn=<ReshapeAliasBackward0>), tensor([[0.1111, 0.7598, 0.3672,  ..., 0.3061, 0.1440, 0.2842],
        [0.2657, 0.0522, 0.2621,  ..., 0.0363, 0.2272, 0.

  0%|          | 11/3055 [00:12<52:11,  1.03s/it]

[tensor([[0.1735, 1.3912, 0.2394,  ..., 0.2795, 0.1643, 0.5283],
        [0.4992, 0.0203, 0.2935,  ..., 0.3883, 0.1765, 4.6640],
        [0.0753, 0.1756, 0.2339,  ..., 1.1095, 0.0155, 0.3600],
        ...,
        [0.7140, 0.5692, 0.4538,  ..., 0.2592, 0.5171, 0.3044],
        [0.1077, 1.2094, 0.1961,  ..., 1.0288, 0.0437, 0.9602],
        [0.0205, 0.5060, 0.9473,  ..., 0.8195, 0.0560, 0.4882]],
       grad_fn=<ReshapeAliasBackward0>), tensor([[0.4931, 0.8447, 0.2776,  ..., 0.5990, 0.2028, 0.2212],
        [0.2844, 0.1453, 0.0804,  ..., 0.1438, 0.0773, 0.2300],
        [0.4367, 0.6660, 0.0899,  ..., 0.0412, 0.2838, 0.1871],
        ...,
        [0.3342, 0.1397, 0.4592,  ..., 0.2362, 0.0444, 0.6778],
        [0.1249, 0.4872, 0.4672,  ..., 0.4119, 0.0045, 0.0271],
        [0.5475, 0.6624, 0.1561,  ..., 0.4537, 0.2599, 0.2959]],
       grad_fn=<ReshapeAliasBackward0>), tensor([[0.1111, 0.7598, 0.3672,  ..., 0.3061, 0.1440, 0.2842],
        [0.2657, 0.0522, 0.2621,  ..., 0.0363, 0.2272, 0.

: 

: 